# Programming Assignment

## Saving and loading models, with application to the EuroSat dataset

### Instructions

In this notebook, you will create a neural network that classifies land uses and land covers from satellite imagery. You will save your model using Tensorflow's callbacks and reload it later. You will also load in a pre-trained neural network classifier and compare performance with it.

Some code cells are provided for you in the notebook. You should avoid editing provided code, and make sure to execute the cells in order to avoid unexpected errors. Some cells begin with the line:

`#### GRADED CELL ####`

Don't move or edit this first line - this is what the automatic grader looks for to recognise graded cells. These cells require you to write your own code to complete them, and are automatically graded when you submit the notebook. Don't edit the function name or signature provided in these cells, otherwise the automatic grader might not function properly. Inside these graded cells, you can use any functions or classes that are imported below, but make sure you don't use any variables that are outside the scope of the function.

### How to submit

Complete all the tasks you are asked for in the worksheet. When you have finished and are happy with your code, press the **Submit Assignment** button at the top of this notebook.

### Let's get started!

We'll start running some imports, and loading the dataset. Do not edit the existing imports in the following cell. If you would like to make further Tensorflow imports, you should add them here.

In [ ]:
#### PACKAGE IMPORTS ####

# Run this cell first to import all required packages. Do not make any imports elsewhere in the notebook

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
import numpy as np
import pandas as pd

# If you would like to make further imports from tensorflow, add them here
from pathlib import Path


2025-11-27 08:38:28.004666: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-27 08:38:28.334959: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764254308.430329    2929 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764254308.463218    2929 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764254308.674836    2929 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# Import the data

#### The EuroSAT dataset

In this assignment, you will use the [EuroSAT dataset](https://github.com/phelber/EuroSAT). It consists of 27000 labelled Sentinel-2 satellite images of different land uses: residential, industrial, highway, river, forest, pasture, herbaceous vegetation, annual crop, permanent crop and sea/lake. For a reference, see the following papers:
- Eurosat: A novel dataset and deep learning benchmark for land use and land cover classification. Patrick Helber, Benjamin Bischke, Andreas Dengel, Damian Borth. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 2019.
- Introducing EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification. Patrick Helber, Benjamin Bischke, Andreas Dengel. 2018 IEEE International Geoscience and Remote Sensing Symposium, 2018.

Your goal is to construct a neural network that classifies a satellite image into one of these 10 classes, as well as applying some of the saving and loading techniques you have learned in the previous sessions.

#### Import the data

The dataset you will train your model on is a subset of the total data, with 4000 training images and 1000 testing images, with roughly equal numbers of each class.
The train and test datasets required for this project can be downloaded from the following links:

`x_train`: https://drive.google.com/open?id=1cUaIEd9-MLJHFGjLz5QziNvfBtYygplX

`y_train`: https://drive.google.com/open?id=1hv24Ufiio9rBeSqgnNoM3dr5sVGwOmBy

`x_test`: https://drive.google.com/open?id=1AH9lKHT5P2oQLz8SGMRPWs_M9wIM2ZRH

`y_test`: https://drive.google.com/open?id=1i4_azocSDuU3TcDf3OSHO1vF0D5-xMU6

You should store these files in Drive for use in this Colab notebook.

In [2]:
# create the new folder weekly_data and download the data from the link provided in the instructions
!mkdir -p weekly_data
!wget -P weekly_data/ https://zenodo.org/api/records/7711810/files-archive

--2025-11-27 08:44:12--  https://zenodo.org/api/records/7711810/files-archive
Resolving zenodo.org (zenodo.org)... 188.185.48.75, 137.138.52.235, 188.185.43.153, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘weekly_data/files-archive’

files-archive           [  <=>               ]   2.01G  1.56MB/s    in 35m 42s 

2025-11-27 09:19:59 (985 KB/s) - ‘weekly_data/files-archive’ saved [2160061446]



In [6]:
!mv weekly_data/files-archive weekly_data/files-archive.zip

In [7]:
zip_path = "weekly_data/files-archive.zip"

In [3]:
# add the folder to gitignore
!echo "weekly_data/" >> .gitignore

In [ ]:
!unzip -l weekly_data/files-archive.zip | head -40

/bin/bash: line 1: unzip: command not found


In [10]:
import zipfile

zip_path = "weekly_data/files-archive.zip"  # or "weekly_data/files-archive.zip"

with zipfile.ZipFile(zip_path, "r") as zf:
    files = zf.namelist()
    print("Number of entries:", len(files))
    for name in files[:40]:
        print(name)

Number of entries: 2
EuroSAT_MS.zip
EuroSAT_RGB.zip


In [11]:
import zipfile
from pathlib import Path

outer_zip = Path("weekly_data/files-archive.zip")  # or .zip if you renamed it
output_dir = Path("weekly_data")
output_dir.mkdir(parents=True, exist_ok=True)

with zipfile.ZipFile(outer_zip, "r") as zf:
    # Extract only the RGB version
    zf.extract("EuroSAT_RGB.zip", path=output_dir)
    # If you also want MS:
    # zf.extract("EuroSAT_MS.zip", path=output_dir)

print((output_dir / "EuroSAT_RGB.zip").exists())  # should be True


True


In [16]:
inner_zip = output_dir / "EuroSAT_RGB.zip"
rgb_dir = output_dir / "EuroSAT_RGB"
rgb_dir.mkdir(parents=True, exist_ok=True)

with zipfile.ZipFile(inner_zip, "r") as zf:
    zf.extractall(rgb_dir)

print("Extracted to:", rgb_dir)

Extracted to: weekly_data/EuroSAT_RGB


In [19]:
# view all the new files in the folder
!ls -lR weekly_data /EuroSAT_RGB | head -40

ls: cannot access '/EuroSAT_RGB': No such file or directory
weekly_data:
total 2201888
drwxr-xr-x 3 nvidia nvidia       4096 Nov 27 09:43 EuroSAT_RGB
-rw-r--r-- 1 nvidia nvidia   94658721 Nov 27 09:43 EuroSAT_RGB.zip
-rw-r--r-- 1 nvidia nvidia 2160061446 Nov 27 09:19 files-archive.zip

weekly_data/EuroSAT_RGB:
total 4
drwxr-xr-x 12 nvidia nvidia 4096 Nov 27 09:43 EuroSAT_RGB

weekly_data/EuroSAT_RGB/EuroSAT_RGB:
total 1168
drwxr-xr-x 2 nvidia nvidia 126976 Nov 27 09:43 AnnualCrop
drwxr-xr-x 2 nvidia nvidia 118784 Nov 27 09:43 Forest
drwxr-xr-x 2 nvidia nvidia 176128 Nov 27 09:43 HerbaceousVegetation
drwxr-xr-x 2 nvidia nvidia  94208 Nov 27 09:43 Highway
drwxr-xr-x 2 nvidia nvidia 110592 Nov 27 09:43 Industrial
drwxr-xr-x 2 nvidia nvidia  69632 Nov 27 09:43 Pasture
drwxr-xr-x 2 nvidia nvidia 131072 Nov 27 09:43 PermanentCrop
drwxr-xr-x 2 nvidia nvidia 135168 Nov 27 09:43 Residential
drwxr-xr-x 2 nvidia nvidia  81920 Nov 27 09:43 River
drwxr-xr-x 2 nvidia nvidia 118784 Nov 27 09:43 SeaLa

# Load the Data

In [ ]:
def load_eurosat_data(
    data_root="weekly_data/EuroSAT_RGB/EuroSAT_RGB",
    image_size=(64, 64),
    batch_size=32,
    fraction=0.05,      # 0.05 = use 5% of images; set to 1.0 for all
    seed=1337,
):
    """
    Returns (x_train, y_train), (x_test, y_test) as NumPy arrays
    built from the EuroSAT RGB directory structure.
    """

    data_root = Path(data_root)

    # 1) Build TF datasets from directory (80% train, 20% "test")
    train_ds = tf.keras.utils.image_dataset_from_directory(
        data_root,
        validation_split=0.2,
        subset="training",
        seed=seed,
        image_size=image_size,
        batch_size=batch_size,
        label_mode="int",
    )

    test_ds = tf.keras.utils.image_dataset_from_directory(
        data_root,
        validation_split=0.2,
        subset="validation",
        seed=seed,
        image_size=image_size,
        batch_size=batch_size,
        label_mode="int",
    )

    # 2) Optionally reduce to a fraction (e.g. 5%) of the images
    if fraction is not None and fraction < 1.0:
        # Estimate total number of examples from number of batches * batch_size
        n_train_batches = tf.data.experimental.cardinality(train_ds).numpy()
        n_test_batches  = tf.data.experimental.cardinality(test_ds).numpy()

        n_train = n_train_batches * batch_size
        n_test  = n_test_batches * batch_size

        n_train_small = int(n_train * fraction)
        n_test_small  = int(n_test * fraction)

        train_ds = train_ds.unbatch().take(n_train_small).batch(batch_size)
        test_ds  = test_ds.unbatch().take(n_test_small).batch(batch_size)

    # 3) Convert tf.data.Dataset → NumPy arrays
    def ds_to_numpy(ds):
        xs, ys = [], []
        for x_batch, y_batch in ds:
            xs.append(x_batch.numpy())
            ys.append(y_batch.numpy())
        x = np.concatenate(xs, axis=0)
        y = np.concatenate(ys, axis=0)
        return x, y

    x_train, y_train = ds_to_numpy(train_ds)
    x_test, y_test   = ds_to_numpy(test_ds)

    return (x_train, y_train), (x_test, y_test)


In [27]:
(x_train, y_train), (x_test, y_test) = load_eurosat_data()

# scale pixels to [0, 1]
x_train = x_train / 255.0
x_test  = x_test  / 255.0

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)


Found 27000 files belonging to 10 classes.
Using 21600 files for training.
Found 27000 files belonging to 10 classes.
Using 5400 files for validation.
(1080, 64, 64, 3) (1080,)
(270, 64, 64, 3) (270,)


# Build the neural network model

You can now construct a model to fit to the data. Using the Sequential API, build your model according to the following specifications:

* The model should use the input_shape in the function argument to set the input size in the first layer.
* The first layer should be a Conv2D layer with 16 filters, a 3x3 kernel size, a ReLU activation function and 'SAME' padding. Name this layer 'conv_1'.
* The second layer should also be a Conv2D layer with 8 filters, a 3x3 kernel size, a ReLU activation function and 'SAME' padding. Name this layer 'conv_2'.
* The third layer should be a MaxPooling2D layer with a pooling window size of 8x8. Name this layer 'pool_1'.
* The fourth layer should be a Flatten layer, named 'flatten'.
* The fifth layer should be a Dense layer with 32 units, a ReLU activation. Name this layer 'dense_1'.
* The sixth and final layer should be a Dense layer with 10 units and softmax activation. Name this layer 'dense_2'.

In total, the network should have 6 layers.

In [ ]:
#### GRADED CELL ####

# Complete the following function.
# Make sure to not change the function name or arguments.

def get_new_model(input_shape):
    """
    This function should build a Sequential model according to the above specification. Ensure the
    weights are initialised by providing the input_shape argument in the first layer, given by the
    function argument.
    Your function should also compile the model with the Adam optimiser, sparse categorical cross
    entropy loss function, and a single accuracy metric.
    """

    model = tf.keras.Sequential()

    # 1) Conv2D, 16 filters, 3x3, ReLU, SAME padding
    model.add(Conv2D(
            filters=16,
            kernel_size=(3, 3),
            activation="relu",
            padding="same",
            input_shape=input_shape,
            name="conv_1",
        ))
    
    # 2) Conv2D, 8 filters, 3x3, ReLU, SAME padding
    model.add(Conv2D(
            filters=8,
            kernel_size=(3, 3),
            activation="relu",
            padding="same",
            name="conv_2",
        ))
    
    # 3) MaxPooling2D, 8x8
    model.add(MaxPooling2D(pool_size=(8, 8), name="pool_1"))
    
    # 4) Flatten
    model.add(Flatten(name="flatten"))
    
    # 5) Dense, 32 units, ReLU
    model.add(Dense(32, activation="relu", name="dense_1"))
    
    # 6) Dense, 10 units, softmax
    model.add(Dense(10, activation="softmax", name="dense_2"))

    # Compile with Adam, sparse categorical crossentropy, accuracy
    optimizer = tf.keras.optimizers.Adam()
    loss = 'sparse_categorical_crossentropy'
    metrics = ['accuracy']

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=metrics,
    )

    return model

#### Compile and evaluate the model

In [37]:
# Run your function to create the model

model = get_new_model(x_train[0].shape)

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_1 (Conv2D)                 │ (None, 64, 64, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2 (Conv2D)                 │ (None, 64, 64, 8)      │         1,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_1 (MaxPooling2D)           │ (None, 8, 8, 8)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │        16,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,354 (71.70 KB)

 Trainable params: 18,354 (71.70 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
# Run this cell to define a function to evaluate a model's test accuracy

def get_test_accuracy(model, x_test, y_test):
    """Test model classification accuracy"""
    test_loss, test_acc = model.evaluate(x=x_test, y=y_test)
    print('accuracy: {acc:0.3f}'.format(acc=test_acc))

In [36]:
# Print the model summary and calculate its initialised test accuracy

get_test_accuracy(model, x_test, y_test)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1259 - loss: 2.3065
accuracy: 0.126


# Create checkpoints to save model during training, with a criterion

You will now create three callbacks:
- `checkpoint_every_epoch`: checkpoint that saves the model weights every epoch during training
- `checkpoint_best_only`: checkpoint that saves only the weights with the highest validation accuracy. Use the testing data as the validation data.
- `early_stopping`: early stopping object that ends training if the validation accuracy has not improved in 3 epochs.

In [ ]:
#### GRADED CELL ####

# Complete the following functions.
# Make sure to not change the function names or arguments.

def get_checkpoint_every_epoch():
    """
    This function should return a ModelCheckpoint object that:
    - saves the weights only at the end of every epoch
    - saves into a directory called 'checkpoints_every_epoch' inside the current working directory
    - generates filenames in that directory like 'checkpoint_XXX' where
      XXX is the epoch number formatted to have three digits, e.g. 001, 002, 003, etc.
    """
    filepath = os.path.join("checkpoints_every_epoch", "checkpoint_{epoch:03d}")
    checkpoint = ModelCheckpoint(
        filepath=filepath,
        save_weights_only=True,
        save_best_only=False,
        save_freq="epoch",
        verbose=0,
    )
    return checkpoint




def get_checkpoint_best_only():
    """
    This function should return a ModelCheckpoint object that:
    - saves only the weights that generate the highest validation (testing) accuracy
    - saves into a directory called 'checkpoints_best_only' inside the current working directory
    - generates a file called 'checkpoints_best_only/checkpoint'
    """
    filepath = os.path.join("checkpoints_best_only", "checkpoint")
    checkpoint = ModelCheckpoint(
        filepath=filepath,
        save_weights_only=True,
        save_best_only=True,
        monitor="val_accuracy",
        mode="max",
        verbose=0,
    )
    return checkpoint


In [39]:
#### GRADED CELL ####

# Complete the following function.
# Make sure to not change the function name or arguments.

def get_early_stopping():
    """
    This function should return an EarlyStopping callback that stops training when
    the validation (testing) accuracy has not improved in the last 3 epochs.
    HINT: use the EarlyStopping callback with the correct 'monitor' and 'patience'
    """
    early_stopping = EarlyStopping(
        monitor="val_accuracy",
        patience=3,
        mode="max",
        verbose=0,
    )
    return early_stopping


In [40]:
# Run this cell to create the callbacks

checkpoint_every_epoch = get_checkpoint_every_epoch()
checkpoint_best_only = get_checkpoint_best_only()
early_stopping = get_early_stopping()

ValueError: When using `save_weights_only=True` in `ModelCheckpoint`, the filepath provided must end in `.weights.h5` (Keras weights format). Received: filepath=checkpoints_every_epoch/checkpoint_{epoch:03d}

# Train model using the callbacks

Now, you will train the model using the three callbacks you created. If you created the callbacks correctly, three things should happen:
- At the end of every epoch, the model weights are saved into a directory called `checkpoints_every_epoch`
- At the end of every epoch, the model weights are saved into a directory called `checkpoints_best_only` **only** if those weights lead to the highest test accuracy
- Training stops when the testing accuracy has not improved in three epochs.

You should then have two directories:
- A directory called `checkpoints_every_epoch` containing filenames that include `checkpoint_001`, `checkpoint_002`, etc with the `001`, `002` corresponding to the epoch
- A directory called `checkpoints_best_only` containing filenames that include `checkpoint`, which contain only the weights leading to the highest testing accuracy

In [ ]:
# Train model using the callbacks you just created

callbacks = [checkpoint_every_epoch, checkpoint_best_only, early_stopping]
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), callbacks=callbacks)

# Create new instance of model and load on both sets of weights

Now you will use the weights you just saved in a fresh model. You should create two functions, both of which take a freshly instantiated model instance:
- `model_last_epoch` should contain the weights from the latest saved epoch
- `model_best_epoch` should contain the weights from the saved epoch with the highest testing accuracy

_Hint: use the_ `tf.train.latest_checkpoint` _function to get the filename of the latest saved checkpoint file. Check the docs_ [_here_](https://www.tensorflow.org/api_docs/python/tf/train/latest_checkpoint).

In [ ]:
#### GRADED CELL ####

# Complete the following functions.
# Make sure to not change the function name or arguments.

def get_model_last_epoch(model):
    """
    This function should create a new instance of the CNN you created earlier,
    load on the weights from the last training epoch, and return this model.
    """
    # Directory where you saved "checkpoint_001", "checkpoint_002", ...
    ckpt_dir = "checkpoints_every_epoch"

    # Get the path of the latest checkpoint
    latest_ckpt = tf.train.latest_checkpoint(ckpt_dir)

    # Load weights into the provided model instance
    model.load_weights(latest_ckpt)

    return model



def get_model_best_epoch(model):
    """
    This function should create a new instance of the CNN you created earlier, load
    on the weights leading to the highest validation accuracy, and return this model.
    """
    # For the "best only" case, we saved all best weights under a fixed prefix
    best_ckpt_path = os.path.join("checkpoints_best_only", "checkpoint")

    # Load weights into the provided model instance
    model.load_weights(best_ckpt_path)

    return model


In [ ]:
# Run this cell to create two models: one with the weights from the last training
# epoch, and one wiht the weights leading to the highest validation (testing) accuracy.
# Verify that the second has a higher validation (testing) accuarcy.

model_last_epoch = get_model_last_epoch(get_new_model(x_train[0].shape))
model_best_epoch = get_model_best_epoch(get_new_model(x_train[0].shape))
print('Model with last epoch weights:')
get_test_accuracy(model_last_epoch, x_test, y_test)
print('')
print('Model with best epoch weights:')
get_test_accuracy(model_best_epoch, x_test, y_test)

# Load, from scratch, a model trained on the EuroSat dataset.

In your workspace, you will find another model trained on the `EuroSAT` dataset in `.h5` format. This model is trained on a larger subset of the EuroSAT dataset and has a more complex architecture. The path to the model is `models/EuroSatNet.h5`. See how its testing accuracy compares to your model!

In [ ]:
#### GRADED CELL ####

# Complete the following functions.
# Make sure to not change the function name or arguments.

def get_model_eurosatnet():
    """
    This function should return the pretrained EuroSatNet.h5 model.
    """
    model = tf.keras.models.load_model("models/EuroSatNet.h5")
    return model


In [ ]:
# Run this cell to print a summary of the EuroSatNet model, along with its validation accuracy.

model_eurosatnet = get_model_eurosatnet()
model_eurosatnet.summary()
get_test_accuracy(model_eurosatnet, x_test, y_test)

Congratulations for completing this programming assignment! You're now ready to move on to the capstone project for this course.